In [69]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment
from azureml.pipeline.core import Pipeline, StepSequence

In [59]:
# Environnement

compute_engine = 'jucalcul1'
cluster_name = 'jucalcul1'
dataset_name = 'trainingdataset'
experiment_name = "25082021"

# if not exist yet
#ws = Workspace.create(name='myworkspace',
#                      subscription_id='<azure-subscription-id>',
#                      resource_group='myresourcegroup',
#                      create_resource_group=True,
#                      location='eastus2')

ws = Workspace.get(subscription_id = subscription_id,
                   resource_group = resource_group,
                   name = workspace_name)

In [60]:
# Compute 
components_dir = '.'
os.makedirs(components_dir, exist_ok=True)


# Verify that the compute cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster : ', cluster_name)
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=3)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex, cluster_name)


Found existing cluster :  jucalcul1


In [61]:
# environment preparation : 
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
aml_run_config.target = compute_engine

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]','pandas','joblib','azureml-core==1.20.0','PyJWT==1.7.1'], 
        pin_sdk_version=False)

In [169]:
# Step 0 environnement
data_folder = os.path.join(os.getcwd(), 'data/')
os.makedirs(data_folder, exist_ok=True)
def_blob_store = ws.get_default_datastore() 

#sklearn_env = Environment.from_conda_specification(name = 'sklearn_envv2', file_path = './conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

# Step 0 data ingestion
raw_ds = Dataset.get_by_name(ws, name=dataset_name, version='latest') 
datastore = ws.get_default_datastore()

# allow to use output1 in input2
fileConfig = OutputFileDatasetConfig(name='file_dataset', destination=(datastore, "mypathdata/{run-id}/{output-name}")).as_upload()


# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = compute_engine,
                         runconfig=aml_run_config,
                         allow_reuse=True,
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'), # has_named_input is the method for tabular dataset 
                                      '--output-dir', fileConfig])

# register clean data preprocess
step1_output_ds = fileConfig.register_on_complete(name='processed_data', 
                                                        description = 'files from step1')
model_file = PipelineData("model_file", datastore=def_blob_store,
                                        pipeline_output_name= 'model_output',
                                        training_output=TrainingOutput(type='Model'))

# Step 2 : training 
train_src = ScriptRunConfig(source_directory = 'scripts',
                            script = 'model_script2.py',
                            compute_target = compute_engine,
                            arguments = ["--input-dir", fileConfig.as_input()],
                            environment = sklearn_env)


# Step 2 : hyperparameter 
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy"),
    "--max_depth": choice(2, 8, 16)
    }
)

hd_config = HyperDriveConfig(run_config=train_src, 
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='precision', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=2, # small nb for testing 
                             max_concurrent_runs=2)


    
hd_step = HyperDriveStep(
    allow_reuse=True,
    name='hyperparameters',
    outputs = [model_file],
    hyperdrive_config=hd_config)



# step 3 : register best model
register_model = PythonScriptStep(name = 'Model Registration',
                                  script_name = 'scripts/register_model2.py',
                                  compute_target = compute_engine,
                                  runconfig=aml_run_config,
                                  allow_reuse=True
                                  )


# step 4 : publish pipeline
pipeline_training = [step1, hd_step, register_model]
pipeline = Pipeline(workspace=ws, steps =StepSequence(steps=pipeline_training))
pipeline_run = pipeline.submit(experiment_name, regenerate_outputs=True)

published_pipeline1 = pipeline.publish(
                        name="Template_Pipeline_Notebook",
                        description="Published Pipeline Description",
                        version="1.0")


# step 5 : schedule pipeline run every day
#recurrence = ScheduleRecurrence(frequency='Day', interval=1)
#recurring_schedule = Schedule.create(ws, name='DailySchedule', 
#                            description='Once a day',
#                            pipeline_id=published_pipeline1.id, 
#                            experiment_name= experiment_name, 
#                            recurrence=recurrence)


Created step prepare data [9d84fabf][571f08b9-d7a0-4e6c-be97-3c51e01f68ea], (This step will run and generate new outputs)
Created step hyperparameters [c54919b0][a727b193-cab6-4a8a-af26-9c4304eeb00d], (This step will run and generate new outputs)
Created step Model Registration [f97e9fb0][b1c4b988-ce70-4d52-8051-241f7567212a], (This step will run and generate new outputs)
Submitted PipelineRun ab7ea8ca-4154-49d5-9e46-0d85b27c3208
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ab7ea8ca-4154-49d5-9e46-0d85b27c3208?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [163]:
pipeline_run.get_details()['properties']
summaryid = [0]
summarymetric = [0]

for child_run in pipeline_run.get_children():
    listid = child_run.id
    print('id parent', listid)
    for  child_run2 in child_run.get_children():
        print('id pipeline step',child_run2)
        for  child_run3 in child_run2.get_children():
            listid = child_run3.id
            summaryid.append(listid)
            list_metric = child_run3.get_metrics()['Test Precison']
            summarymetric.append(list_metric)

print(summarymetric)
print(summaryid)
maxi = summarymetric.index(max(summarymetric))
maxi_id = summaryid[maxi]
print('best model id : ', maxi_id)


print('start register model')
for child_run3 in child_run2.get_children():
    if child_run3.id == maxi_id:
        child_run3.register_model(model_name='rf_tuning', model_path='outputs_here/model.pkl')
        print("ok it's saved")



[0]
[0]
best model id :  0
start register model
